In [1]:
from segmenter_model_zoo.zoo import SegModel
from aicsimageio import AICSImage
from itkwidgets import view
import numpy as np
from aicsmlsegment.utils import background_sub, simple_norm

## step 1: define a basic model object

In [2]:
my_model = SegModel()

In [3]:
# you can check all avaialable models:
my_model.list_all_trained_models()

DNA_mask_production
DNA_seed_production
CellMask_edge_production
CAAX_production
LMNB1_all_production
LMNB1_fill_production
LMNB1_core_production
LF_DNA_mask
LF_DNA_mask_two_camera
LF_mem_edge
LF_mem_edge_two_camera
H2B_coarse


## step 2: load one pre-trained model

Suppose you want to load the "DNA_mask_production". You can do `my_model.load_train('DNA_mask_production')`. This will automatically download the pre-trained model from cloud storage and save a local copy at "./". You can also specify where to save the local copy by, for example, `my_model.load_train('DNA_mask_production', {"local_path":"./all_models/"})`. If you already have downloaded the model before and saved it at "./all_models/", there won't be any download and the local model will be directly loaded.

In [4]:
my_model.load_train('DNA_mask_production', {"local_path":"./"})
my_model.to_gpu('cuda:0')

model DNA_mask_production is successfully loaded


## step 3: apply the model on one image

There are two different ways to apply the model: applying on a numpy array or just providing a filepath and which channel to load.

In [5]:
# read one image as numpy array, and apply the model on it
reader = AICSImage("C:/projects/demo/data/3500000875_100X_20170508_12-Scene-3-P37-E06.ome.tiff")
dna_img = reader.get_image_data("CZYX", C=[-2], S=0, T=0)
dna_mask = my_model.apply_on_single_zstack(dna_img)

subtracted background


In [6]:
# The output is not binary by default. The value is between 0 and 1, indicating the probability of 
# being object/ Bindary results can be obtained by a cutoff value
dna_mask_bw = dna_mask>0.7
dna_mask_bw = dna_mask_bw.astype(np.uint8)
dna_mask_bw[dna_mask_bw>0]=255

In [7]:
view(dna_mask_bw)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [8]:
# or we can just provide the filename and which channel to use. 
# We can also pass in the cutoff so the returned value is already binary
fn = "C:/projects/demo/data/3500000875_100X_20170508_12-Scene-3-P37-E06.ome.tiff"
dna_mask_bw = my_model.apply_on_single_zstack(filename=fn, inputCh=-2, cutoff=0.7)

subtracted background


In [9]:
view(dna_mask_bw)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

### Note:

The model was developed for working on images from the Allen Cell Data Collection. When applying on your own data, it may or may not work well. If not good, what to do? 

First, our data was built for 100x ZEISS spinning disk images. The physical size of each voxel is 0.108 um x 0.108 um x 0.29 um (XYZ). If you image is of different magnification, try resize your image before applying the model.

Second, you can try different image normalization methods. You can check out which normalization method each model is using (e.g. [here](https://github.com/AllenCell/segmenter_model_zoo/blob/main/segmenter_model_zoo/zoo.py#L43) ), check the exact implementation of each normalization method [here](https://github.com/AllenInstitute/aics-ml-segmentation/blob/master/aicsmlsegment/utils.py#L98). Then, you can pick one image from our data and apply the normalization method the model is using and see how it looks. Then, you may adjust the normalization method to be applied on your data so that the normalized image is comparable to the normalization results on our data, in terms of contrast, brightness, etc.. Finally, you can load your image, do your new normalization on image, apply the model like this `my_model.apply_on_single_zstack(dna_img, already_normalized, already_normalized=True)` This is bypass any normalizaiton the model does by default.

Finally, it is possible that the models won't work off-the-shelf. That's okay. The essential philosophy behind the iterative deep learning workflow is to start with some preliminary results and gradually improve the performance. So, try the two adjustments above about magnification and normalization in order to make the results as good as possible. Then, if you can find just a few cells where the segmentation is acceptable, then you can use the curation tool in our Segmenter (see [here](https://github.com/AllenInstitute/aics-ml-segmentation/blob/master/docs/bb2.md)) to collect a few good examples to finetune the model on your data. Then, when applying the new model on your data, the results may be better. You may need one more iteration of curation + training. 